<a href="https://colab.research.google.com/github/arekciminski/model_SW_SNN/blob/Dane_do_duzej_sieci_v2/SW_Logistic_KNN_DT_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ładowanie bibliotek 

In [4]:
import os.path
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score


#Ładowanie pliku z danymi z GitHub'a

In [5]:
#file_name='dane_do_uczenia_1_1000000.zip'
'''if 
os.path.exists(file_name)==0:
  from google.colab import drive
  drive.mount("/content/drive/dane_do_uczenia_1_1000000.zip")'''
#!unzip "dane_do_uczenia_1_1000000.zip"


'if \nos.path.exists(file_name)==0:\n  from google.colab import drive\n  drive.mount("/content/drive/dane_do_uczenia_1_1000000.zip")'

In [6]:
file_name='dane_do_uczenia_1_50000.csv'
#https://raw.githubusercontent.com/arekciminski/model_SW_SNN/Dane_do_duzej_sieci_v2/dane_do_uczenia_1_50000.csv
if os.path.exists(file_name)==0:
  !rm -rf file_name
  #!wget "https://raw.githubusercontent.com/arekciminski/model_SW_SNN/Dane_do_duzej_sieci_v2/dane_do_uczenia_4000000.csv"
  !wget 'https://raw.githubusercontent.com/arekciminski/model_SW_SNN/Dane_do_duzej_sieci_v2/dane_do_uczenia_1_50000.csv'

--2023-05-17 20:39:52--  https://raw.githubusercontent.com/arekciminski/model_SW_SNN/Dane_do_duzej_sieci_v2/dane_do_uczenia_1_50000.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12601058 (12M) [text/plain]
Saving to: ‘dane_do_uczenia_1_50000.csv’

dane_do_uczenia_1_5 100%[===================>]  12.02M  --.-KB/s    in 0.06s   

2023-05-17 20:39:52 (190 MB/s) - ‘dane_do_uczenia_1_50000.csv’ saved [12601058/12601058]



#Ładowanie pliku lokalnego do pamięci


In [7]:
df= pd.read_csv(file_name,decimal =',',sep=';')

In [8]:
df.shape

(50000, 22)

In [9]:
df.head()

,Unnamed: 0,error_output,time_output,head_output_001,head_output_002,head_output_055,head_output_083,head_output_158,head_output_180,flow_output_F1,...,tank_output_180,energy_output_F1,energy_output_K1,energy_output_P1,tank_input_180,pump_input_F1,pump_input_K1,pump_input_P1,demand_input_par2,demand_input_kar2
0,0,4,0,171.000000,202.688492,202.813400,202.669708,202.419632,171.000000,0.000000,...,171.000000,0.000000,0.000000,9.551300,5.0,0.69,0.51,0.85,0.30,0.38
1,1,0,0,174.102203,185.410309,185.258530,184.965164,184.823532,168.100006,249.311569,...,168.412277,42.588947,9.225259,3.575605,2.1,0.91,0.57,0.66,0.30,1.17
2,2,4,0,170.899994,195.247971,195.357498,195.042892,194.887009,170.899994,0.000000,...,170.731049,0.000000,6.972664,10.140953,4.9,0.64,0.63,0.88,0.50,0.97
3,3,4,0,180.594803,209.881119,209.265900,209.150177,208.931396,171.000000,0.000092,...,170.514648,0.000036,35.224850,0.000000,5.0,0.80,0.86,0.54,0.79,1.03
4,4,4,0,167.755081,213.000137,213.120621,212.982132,212.730011,167.300003,61.925354,...,167.455750,8.200216,0.000000,12.840910,1.3,0.67,0.65,0.95,0.34,0.37


#Losowa zmiana pozycji wierszy w df


In [10]:
df = df.sample(frac = 1)

In [11]:
df['error_output'].value_counts()

4    33283
0    16709
6        8
Name: error_output, dtype: int64

#Podział danych na X i y

In [12]:
df.columns

Index(['Unnamed: 0', 'error_output', 'time_output', 'head_output_001',
       'head_output_002', 'head_output_055', 'head_output_083',
       'head_output_158', 'head_output_180', 'flow_output_F1',
       'flow_output_K1', 'flow_output_P1', 'tank_output_180',
       'energy_output_F1', 'energy_output_K1', 'energy_output_P1',
       'tank_input_180', 'pump_input_F1', 'pump_input_K1', 'pump_input_P1',
       'demand_input_par2', 'demand_input_kar2'],
      dtype='object')

In [13]:
y = df['error_output'].replace([4,6],[1,1])

In [14]:
x_columns =['tank_input_180',  'head_output_001',
            'head_output_002', 'head_output_055', 'head_output_083',
            'head_output_158', 'head_output_180', 'pump_input_P1',
            'demand_input_par2', 'demand_input_kar2']
y_column = ['energy_output_F1', 'energy_output_K1', 'energy_output_P1',
            'pump_input_F1', 'pump_input_K1', 'pump_input_P1','error_output',
            'energy_output_F1', 'energy_output_K1', 'energy_output_P1',]

In [15]:
X = df[x_columns]
y = df[y_column]
X

,tank_input_180,head_output_001,head_output_002,head_output_055,head_output_083,head_output_158,head_output_180,pump_input_P1,demand_input_par2,demand_input_kar2
33093,4.3,174.897110,189.151581,188.951202,188.743011,188.558868,170.300003,0.61,0.76,1.02
18906,1.8,168.553635,187.531769,187.667374,187.288086,187.164017,167.800003,0.87,1.33,1.08
39201,4.5,177.828888,180.642822,180.338257,180.018982,179.902740,170.500000,0.55,0.58,1.31
3767,2.2,174.192123,212.893250,212.393265,212.021957,211.945633,168.199997,0.91,0.29,1.49
33168,4.2,170.199997,183.859604,183.461166,183.138107,183.029282,170.199997,0.55,0.67,1.36
...,...,...,...,...,...,...,...,...,...,...
23702,4.1,170.100006,203.158234,202.964462,202.938080,202.657089,170.100006,0.58,0.40,0.46
45263,1.4,167.399994,199.540802,199.821518,199.489960,199.317352,167.399994,1.00,1.49,0.68
31933,3.9,174.743820,192.171646,192.014450,191.994797,191.709595,169.899994,0.56,1.13,0.39
22118,2.1,173.912277,195.633850,195.689224,195.231659,195.159866,168.100006,0.96,1.18,1.31


#Podział danych na testowe i treningowe

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [17]:
dt_reg = DecisionTreeRegressor(random_state=0)


In [18]:
dt_reg.fit(X_train,y_train)

DecisionTreeRegressor(random_state=0)

In [19]:
y_pred_LR = dt_reg.predict(X_test)
y_scores_LR = dt_reg.predict(X_test)

In [20]:
MSE_test_LR = mean_squared_error(y_test, y_pred_LR)

print(f"Accuracy:{MSE_test_LR:0.4f}" )


Accuracy:6.0470


In [21]:
cross_val_score(dt_reg, X_train, y_train,cv=10)

array([0.87315167, 0.87420321, 0.88046406, 0.87450985, 0.87783263,
       0.87969253, 0.88446391, 0.87168926, 0.87861505, 0.87189075])

In [28]:
param_grid_dt ={
    'max_depth' : list(range(2,21,2)),
    'min_samples_split' : list(range(2,21,2)),
}

In [33]:
dt_reg_GS = GridSearchCV(dt_reg, param_grid_dt,verbose=2,n_jobs=1)

dt_reg_GS.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=6; total time=   0.1s
[CV] END ...................max_depth=2, min_s

GridSearchCV(estimator=DecisionTreeRegressor(random_state=0), n_jobs=1,
             param_grid={'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
                         'min_samples_split': [2, 4, 6, 8, 10, 12, 14, 16, 18,
                                               20]},
             verbose=2)

In [30]:
y_pred = dt_reg_GS.predict(X_test)

In [31]:
MSE_test_LR = mean_squared_error(y_test, y_pred)
MSE_test_LR

5.908118260374364

In [32]:
from joblib import dump, load
dump(dt_reg_GS, 'DT_best_pred_model.joblib')

['DT_best_pred_model.joblib']